In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split

In [3]:
data_path = '/kaggle/input/ham10000-preprocessed-dataset/balanced and standardized images meta for custom.csv'
image_folder = '/kaggle/input/ham10000-preprocessed-dataset/balanced_and_standardized_images/balanced_and_standardized_images'
df = pd.read_csv(data_path)

# Define parameters
IMG_SIZE = 96
BATCH_SIZE = 32
LEARNING_RATE = 0.0001
DROPOUT_RATE = 0.5
EPOCHS = 20

# Get unique classes and create label mapping
classes = df['dx'].unique()
class_to_idx = {c: i for i, c in enumerate(classes)}
df['label'] = df['dx'].map(class_to_idx)



In [4]:
# Train-test split
train_df, test_df = train_test_split(df, test_size=0.2, stratify=df['label'], random_state=42)

In [5]:
# Image Data Generators with Augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # 20% of train set will be used as validation
)

train_generator = train_datagen.flow_from_dataframe(
    train_df, directory=image_folder, x_col='image_id', y_col='dx',
    target_size=(IMG_SIZE, IMG_SIZE), batch_size=BATCH_SIZE,
    class_mode='categorical', subset='training'
)

val_generator = train_datagen.flow_from_dataframe(
    train_df, directory=image_folder, x_col='image_id', y_col='dx',
    target_size=(IMG_SIZE, IMG_SIZE), batch_size=BATCH_SIZE,
    class_mode='categorical', subset='validation'
)

test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_dataframe(
    test_df, directory=image_folder, x_col='image_id', y_col='dx',
    target_size=(IMG_SIZE, IMG_SIZE), batch_size=BATCH_SIZE,
    class_mode='categorical', shuffle=False
)

Found 30039 validated image filenames belonging to 7 classes.
Found 7509 validated image filenames belonging to 7 classes.
Found 9387 validated image filenames belonging to 7 classes.


In [6]:
# Load MobileNetV2 base model
base_model = MobileNetV2(input_shape=(IMG_SIZE, IMG_SIZE, 3), include_top=False, weights='imagenet')
base_model.trainable = False  # Freeze base model

# Build custom classification head
x = GlobalAveragePooling2D()(base_model.output)
x = Dropout(DROPOUT_RATE)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(DROPOUT_RATE)(x)
output_layer = Dense(len(classes), activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output_layer)

# Compile the model
model.compile(optimizer=Adam(learning_rate=LEARNING_RATE),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Early stopping callback
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [7]:
# Train the model
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=EPOCHS,
    callbacks=[early_stop]
)

Epoch 1/20


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


939/939 ━━━━━━━━━━━━━━━━━━━━ 579s 607ms/step - accuracy: 0.2139 - loss: 2.4127 - val_accuracy: 0.3704 - val_loss: 1.6852
Epoch 2/20
939/939 ━━━━━━━━━━━━━━━━━━━━ 414s 439ms/step - accuracy: 0.3220 - loss: 1.7170 - val_accuracy: 0.4082 - val_loss: 1.5649
Epoch 3/20
939/939 ━━━━━━━━━━━━━━━━━━━━ 398s 423ms/step - accuracy: 0.3569 - loss: 1.6235 - val_accuracy: 0.4357 - val_loss: 1.4945
Epoch 4/20
939/939 ━━━━━━━━━━━━━━━━━━━━ 392s 416ms/step - accuracy: 0.3788 - loss: 1.5623 - val_accuracy: 0.4521 - val_loss: 1.4377
Epoch 5/20
939/939 ━━━━━━━━━━━━━━━━━━━━ 379s 402ms/step - accuracy: 0.3956 - loss: 1.5207 - val_accuracy: 0.4665 - val_loss: 1.3904
Epoch 6/20
939/939 ━━━━━━━━━━━━━━━━━━━━ 385s 408ms/step - accuracy: 0.4054 - loss: 1.4953 - val_accuracy: 0.4824 - val_loss: 1.3758
Epoch 7/20
939/939 ━━━━━━━━━━━━━━━━━━━━ 385s 408ms/step - accuracy: 0.4263 - loss: 1.4491 - val_accuracy: 0.4911 - val_loss: 1.3572
Epoch 8/20
939/939 ━━━━━━━━━━━━━━━━━━━━ 427s 392ms/step - accuracy: 0.4219 - loss: 1.45

In [8]:
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
import seaborn as sns

def plot_model_training_curve(history):
    fig = make_subplots(rows=1, cols=2, subplot_titles=['Model Accuracy', 'Model Loss'])
    fig.add_trace(
        go.Scatter(
            y=history.history['accuracy'], 
            name='train_acc'), 
        row=1, col=1)
    fig.add_trace(
        go.Scatter(
            y=history.history['val_accuracy'], 
            name='val_acc'), 
        row=1, col=1)
    fig.add_trace(
        go.Scatter(
            y=history.history['loss'], 
            name='train_loss'), 
        row=1, col=2)
    fig.add_trace(
        go.Scatter(
            y=history.history['val_loss'], 
            name='val_loss'), 
        row=1, col=2)
    fig.show()

In [9]:
plot_model_training_curve(history)

In [10]:
# Test function
def evaluate_model():
    test_loss, test_acc = model.evaluate(test_generator)
    print(f'\nTest Accuracy: {test_acc * 100:.2f}%')

# Run evaluation
evaluate_model()

/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning:

Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.



294/294 ━━━━━━━━━━━━━━━━━━━━ 121s 412ms/step - accuracy: 0.4869 - loss: 1.2903

Test Accuracy: 49.37%


In [11]:
# Saving the model
model.save('/kaggle/working/MobileNetV2.keras')
model.save_weights('/kaggle/working/MobileNetV2_weights.weights.h5')